# ETL Pipeline: Transform

## Imports

In [1]:
import sys

sys.path.insert(1, '../..')

import datetime
import logging

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

os.environ['XDG_CACHE_HOME']='Z:\AI_Models'
model_path = 'Z:\AI_Models\huggingface\hub\models--microsoft--Phi-3-mini-4k-instruct\snapshots\c1358f8a35e6d2af81890deffbbfa575b978c62f'

from src.config.config import Config
from src.database.chroma_manager import ChromaManager
from src.database.database import Database
from src.load.data_loader import DataLoader
from src.transform.email_summary import summarize_messages
from src.transform.llm_invoker import LLMInvoker
from src.transform.message_classification import classify_categories
from src.transform.product_classification import classify_products
from src.transform.ner import extract_entities_from_messages
from src.transform.spam_classification import (
    classify_spam_messages_with_llm,
    zero_shot_classify_spam_messages,
)
from src.transform.topic_modelling import TopicModellor
from src.utils.checkpoint import DataFrameCheckpointer

logging.basicConfig(level=logging.INFO)
config = Config.from_json("../../config.json")
llm_invoker = LLMInvoker(model_name=model_path, use_ollama=config.use_ollama)
database = Database.from_credentials(username=config.db_user, password=config.db_password, host=config.db_host, database=config.db_name)
loader = DataLoader(database)
                         
DATA_DIR = '../../data'
PST_DIR = config.pst_directory
DATE = datetime.datetime.now().strftime("%Y-%m-%d")

checkpointer = DataFrameCheckpointer(DATA_DIR + '/checkpoints')

/home/belalm-linux/miniconda3/envs/qib/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/belalm-linux/miniconda3/envs/qib/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/home/belalm-linux/miniconda3/envs/qib/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, an

In [11]:
loader.clear_all_data()
loader.create_tables()

INFO:root:All tables have been dropped from the database.
INFO:root:All tables have been dropped from the database.
INFO:root:All tables have been created in the database.


## Transformations

### Retrieve from Checkpoint

In [3]:
# df = pd.read_csv(f"{DATA_DIR}/interim/sample_preprocessed_messages_2024-08-21.csv") # Sample dataset: 500 emails
df = pd.read_csv(f"{DATA_DIR}/interim/preprocessed_messages_2024-09-01.csv") # Full dataset

C:\ProgramData\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3505: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### Retrieve Quarterly, Monthly, and Weekly Sets of Messages from DB

### Filter Emails

Most feature engineering tasks don't need to be run on all emails. The following feature engineering tasks are intended for customer oriented emails. We can safely disregard internal emails and outgoing emails.

In [5]:
message_df = df.loc[(df["is_internal"] == False) & (df["from_address"] != "info@qib.com.qa")]

In [6]:
message_df = df.dropna(subset=['message_id'])

In [7]:
len(message_df)

64933

In [8]:
message_df = message_df.drop_duplicates(subset=['message_id'])

In [10]:
message_df["topic_id"]=-1
message_df["is_spam"]=False

In [11]:
import numpy as np

In [13]:
message_df["is_internal"]=message_df["sender_domain"].apply(
        lambda x: "qib" in x #any("qib" in domain for domain in str(x).split(", "))
    )

In [18]:
message_df = message_df.where(pd.notnull(message_df), None)
message_df=message_df.replace({np.nan:None})

In [19]:
message_df.submit_time=pd.to_datetime(message_df.submit_time).dt.tz_localize(None)
message_df.delivery_time=pd.to_datetime(message_df.delivery_time).dt.tz_localize(None)

In [ ]:
old_df = message_df.copy()

In [ ]:
message_df = old_df[:2000]

In [ ]:
message_df=df.copy()

#### Spam Classification

Further filter by removing spam emails.

In [ ]:
# spam_df = classify_spam_messages_with_llm(message_df, llm_invoker)
spam_df = zero_shot_classify_spam_messages(message_df)

In [ ]:
# checkpointer.save("spam_classification", spam_df)

In [ ]:
message_df = message_df.merge(spam_df, on="message_id")
message_df = message_df.loc[message_df["is_spam"] == False]

In [ ]:
message_df["is_spam"]=False

In [ ]:
checkpointer.save("spam_classified_messages", message_df)

In [ ]:
message_df = checkpointer.pull("spam_classified_messages")

In [ ]:
message_df.head()

### Vectorization of Emails

Setup Sentence Transformer and ChromaDB

In [ ]:
chroma = ChromaManager("message_embeddings", model_name=config.embedding_model_name)

Get or Create Sentence Embeddings

In [ ]:
message_df = chroma.populate_embeddings(message_df[:50])

In [ ]:
checkpointer.save("message_embeddings", message_df)

In [ ]:
# message_df = checkpointer.pull("message_embeddings")

### Feature Engineering and Modelling

#### Intent Analysis 

In [ ]:
topic_modellor = TopicModellor(message_df, llm_invoker)
topic_df = topic_modellor.topic_df

In [ ]:
topics_to_describe = topic_df[topic_df["topic_id"] != -1].groupby("topic_id").filter(lambda x: len(x) >= 5)

In [ ]:
topic_df

In [ ]:
topic_descriptions = topic_modellor.get_topic_descriptions(topics_to_describe, llm_invoker)[["topic_id", "description"]]

In [ ]:
checkpointer.save("topic_descriptions", topic_descriptions)

In [ ]:
message_df = topic_df[["message_id", "topic_id"]].merge(message_df, on="message_id")
topics_df = topic_df.merge(topic_descriptions, on="topic_id")[["topic_id", "description"]]

In [ ]:
message_df["topic_id"]=-1

In [ ]:
word_frequencies = topic_modellor.get_topic_word_frequencies(topic_df)[["topic_id", "word", "frequency"]]

In [ ]:
checkpointer.save("topics", topic_df)
checkpointer.save("word_frequencies", word_frequencies)
checkpointer.save("topic_messages", message_df)

Top 10 Clusters, their Descriptions, and their Sizes

In [ ]:
topics_df.head(10)

#### Message Classification

In [ ]:
class_df = classify_categories(message_df)
checkpointer.save("classification", class_df)

#### Product Classification

In [ ]:
product_df = classify_products(message_df)
checkpointer.save("products", product_df)

#### Named Entity Recognition

In [ ]:
entities_df = extract_entities_from_messages(message_df, llm_invoker, use_regex=True)
checkpointer.save("entities", entities_df)

#### Email Summarization

In [ ]:
summary_df = summarize_messages(message_df, llm_invoker)
checkpointer.save("summaries", summary_df)

In [ ]:
message_df.head()

### Final DataFrames

Separate list-like columns into new dataframes

In [ ]:
def create_address_df(df: pd.DataFrame) -> pd.DataFrame:
    def split_addresses(addresses):
        return addresses.split(",") if addresses else []

    # Explode each address type into separate rows
    from_df = pd.DataFrame({
        "message_id": df["message_id"],
        "address_type": "from",
        "address": df["from_address"]
    })

    to_df = df[["message_id", "to_address"]].assign(address_type="to")
    to_df = to_df.explode("to_address").rename(columns={"to_address": "address"})

    cc_df = df[["message_id", "cc_address"]].assign(address_type="cc")
    cc_df = cc_df.explode("cc_address").rename(columns={"cc_address": "address"})

    bcc_df = df[["message_id", "bcc_address"]].assign(address_type="bcc")
    bcc_df = bcc_df.explode("bcc_address").rename(columns={"bcc_address": "address"})

    # Combine all address types into a single dataframe
    address_df = pd.concat([from_df, to_df, cc_df, bcc_df], ignore_index=True)

    return address_df

In [ ]:
def create_reference_df(df: pd.DataFrame) -> pd.DataFrame:
    return df[["message_id", "references"]].explode("references").rename(columns={"references": "reference_message_id"})

In [ ]:
def create_domain_df(df: pd.DataFrame) -> pd.DataFrame:
    return df[["message_id", "sender_domain"]].explode("sender_domain")

In [ ]:
address_df = create_address_df(message_df)
reference_df = create_reference_df(message_df)
domain_df = create_domain_df(message_df)

checkpointer.save("addresses", address_df)
checkpointer.save("references", reference_df)
checkpointer.save("domains", domain_df)

### Exporting Dataframes

In [ ]:
message_df.to_csv(config.output_directory + f"/messages_{DATE}.csv", index=False)
address_df.to_csv(config.output_directory + f"/addresses_{DATE}.csv", index=False)
reference_df.to_csv(config.output_directory + f"/references_{DATE}.csv", index=False)
domain_df.to_csv(config.output_directory + f"/domains_{DATE}.csv", index=False)
word_frequencies.to_csv(config.output_directory + f"/word_frequencies_{DATE}.csv", index=False)
topics_df.to_csv(config.output_directory + f"/topics_{DATE}.csv", index=False)
class_df.to_csv(config.output_directory + f"/classification_{DATE}.csv", index=False)
product_df.to_csv(config.output_directory + f"/products_{DATE}.csv", index=False)
entities_df.to_csv(config.output_directory + f"/entities_{DATE}.csv", index=False)
summary_df.to_csv(config.output_directory + f"/summaries_{DATE}.csv", index=False)

### Load

In [9]:
# # Test Database
# message_df = pd.read_csv('../../data/test/messages.csv')
# address_df = pd.read_csv('../../data/test/addresses.csv')
# reference_df = pd.read_csv('../../data/test/references.csv')
# domain_df = pd.read_csv('../../data/test/domains.csv')
# word_frequencies = pd.read_csv('../../data/test/word_frequencies.csv')
# topics_df = pd.read_csv('../../data/test/topics.csv')
# class_df = pd.read_csv('../../data/test/classifications.csv')
# product_df = pd.read_csv('../../data/test/products.csv')
# entities_df = pd.read_csv('../../data/test/entities.csv')
# summary_df = pd.read_csv('../../data/test/summaries.csv')

In [12]:
loader.load_dataframe(message_df.replace({np.nan: None}), "messages")
loader.load_dataframe(address_df.replace({np.nan: None}), "addresses")
loader.load_dataframe(reference_df.replace({np.nan: None}), "references")
loader.load_dataframe(domain_df.replace({np.nan: None}), "domains")
loader.load_dataframe(word_frequencies.replace({np.nan: None}), "word_frequencies")
loader.load_dataframe(topics_df.replace({np.nan: None}), "topics")
loader.load_dataframe(class_df.replace({np.nan: None}), "classifications")
loader.load_dataframe(product_df.replace({np.nan: None}), "products")
loader.load_dataframe(entities_df.replace({np.nan: None}), "entities")
loader.load_dataframe(summary_df.replace({np.nan: None}), "summaries")

3it [00:00, 52.53it/s]
